
## Settings

In [1]:
from google.colab import drive
drive.mount('/content/drive')#,force_remount=True

Mounted at /content/drive


In [2]:
%reset -f

In [3]:
%cd /content/drive/MyDrive/permanent/RECHERCHE2/ml_flux_euler/

/content/drive/MyDrive/permanent/RECHERCHE2/ml_flux_euler


In [11]:
from Euler.backend import K,Ks
from Euler.initial_conditions import GenParam,FuncGenerator
import Euler.core_solver as core
from Euler.backend import K
from Euler.param import Param,Projecter
import matplotlib.pyplot as plt


In [12]:
import tensorflow as tf

## Test core d'Euler

Attention au pycache: les premier run ne sont pas significatifs. 


### Théorie

* $\rho(x,t)$ : une densité de gaz neutre. 
* $E$ : Energie totale du gaz
* $V$ : vitesse moyenne du gaz

Les inconnues sont 
* $\rho$
* $V$ (ou $\rho V$)
* $p$ (ou $E$).

Le système d'équation est


$\partial_t \rho + \partial_x (\rho V) = 0$


$\partial_t \rho V + \partial_x (\rho V^2 +p)   =0$


$\partial_t E +\partial_x (EV + pV)  =0$

$p$ et $E$ sont équivalente par la formule suivante:

$E = \frac 12 \rho V^2+ \frac 1{\gamma - 1 } p$

$\gamma$ est un paramètre physique. Entre $1$ et $3$.



### Compute solutions

In [18]:
order = 1
nx = 1200 // order
if order == 1:
  nx = 600
nx_ratio = nx // 100

param = Param(nx=nx,batch_size=64,nx_ratio=nx_ratio,
              BC_solver=Param.BC_periodic,
              BC_model=Param.BC_periodic)

k_tf32=K("tf",32)

genParam=GenParam(param,kind=GenParam.kind_changing)
genParam.continuousPart_scale = 0.5
genParam.discontinuousPart_scale =0.5
genParam.sod_scale=0.5
# genParam.initial_speed_factor= 1
genParam.initial_speed_factor= 0
genParam.changing_discountMinSpacing= 0.4
genParam.changing_discountMaxSpacing= 0.8
generator=FuncGenerator(genParam, param, k_tf32)


print("Boundary condition",param.BC_solver)
print("Nombre de points d'espace",param.nx)
print("Interval de travail:",param.xmin,"->",param.xmax)
print("Rapport dt/dx:",param.dt_over_dx)

Param initialised with, nx=600, nx_coarse=99, batch_size=64
Boundary condition periodic
Nombre de points d'espace 600
Interval de travail: 0.0 -> 1.0
Rapport dt/dx: 0.1


In [20]:
nb_t=100

In [21]:
W_train, Y_train = core.compute_fine_solutions(genParam,param, nb_t,order=order,HLLC=True,same_discontinuity_locations=True)

durée du calcul de la solution fine: 0.5169363021850586

### Calcul de Flux

In [8]:
def test_compute_solutions(param:Param,k:K,nb_t=10):
    W=core.init_W(param,k)
    res = [] #
    for t in range(nb_t):
        res.append(W)
        var = core.Var_burger(W, param,k)
        Fnum = core.Flux_HLL(var, param,k)
        W= W - param.dt_over_dx * (Fnum[:, 1:, :] - Fnum[:, :-1, :])

    res=k.stack(res)
    return res

In [9]:
def plot_solutions():
    nb_t=1000
    res=test_compute_solutions(Param(),K("tf",64),nb_t)

    fig, ax = plt.subplots(3, 1)

    for t in range(0, nb_t, nb_t // 10):
        color = "r" if t == 0 else "k"
        alpha = 1 if t == 0 else t / nb_t
        ax[0].set_title("rho")
        ax[0].plot(res[t, 0, :, 0], color, alpha=alpha)
        ax[1].set_title("rhoV")
        ax[1].plot(res[t, 0, :, 1], color, alpha=alpha)
        ax[2].set_title("E")
        ax[2].plot(res[t, 0, :, 2], color, alpha=alpha)

    fig.tight_layout()

In [10]:
plot_solutions()

Param initialised with, nx=2000, nx_coarse=199, batch_size=25


AttributeError: ignored

### Avec numpy

In [ ]:
def test_init_W(k:K):
    param=Param()
    core.init_W(param,k)

In [ ]:
Ks.compare_np_tf_at(test_init_W)

In [ ]:
def test_Var(k:K):
    param=Param(10_000,100)
    W=core.init_W(param,k)
    var=core.Var_burger(W,param,k)

In [ ]:
Ks.compare_np_tf_at(test_Var)

In [ ]:
def test_flux_HLL(k:K):
    param=Param(10_000,100)
    W=core.init_W(param,k)
    var=core.Var_burger(W,param,k)
    Fnum=core.Flux_HLL(var,param,k)

In [ ]:
Ks.compare_np_tf_at(test_flux_HLL)

In [ ]:
Ks.compare_np_tf_at(lambda k:test_compute_solutions(Param(),k))

### Sans numpy

Cela plante avec `batch_size=500`

In [ ]:
Ks.compare_tf_at(lambda k: test_compute_solutions(Param(20_000,100),k))

In [ ]:
Ks.compare_tf(lambda k: test_compute_solutions(Param(20_000,500),k))

## Find the optimal batch_size

In [ ]:
import timeit
import matplotlib.pyplot as plt

In [ ]:
timeit.timeit(lambda :"toto",number=1)

In [ ]:
def look_batch_size(nx):
    perf_32=[]
    perf_64=[]
    batch_sizes=[50,100,200,400,600]
    for batch_size in batch_sizes:

        perf_32.append(timeit.timeit(lambda:test_compute_solutions(Param(nx,batch_size),K("tf",32)),number=1)/batch_size)
        perf_64.append(timeit.timeit(lambda:test_compute_solutions(Param(nx,batch_size),K("tf",64)),number=1)/batch_size)

    plt.plot(batch_sizes,perf_32,label="32")
    plt.plot(batch_sizes,perf_64,label="64")
    plt.legend()


In [ ]:
look_batch_size(5_000)

In [ ]:
look_batch_size(10_000)

In [ ]:
look_batch_size(15_000)

In [ ]:
look_batch_size(20_000)